# Model Data Comparison and Ensemble Modeling Examples
One can easily compare results from multiple models by simply running the chosen flythrough function for the same trajectory and times for different model outputs. Blocks 1-4 show this process using the RealFlight function. You can easily plug in other flythrough functions by copying the code from the chosen function's notebook into block 1. Use the START_HERE notebook to walk you through how to choose the best model and model datasets available on your machine. See the InputOutputDemo notebook for more information on what can be done once the results are functionalized.
The relevant data is then retrieved via CDAWeb in block 5 and used in function composition to compare with the average of the model data results.

In [ ]:
# Run the flythrough for the desired models and variable(s).
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt

# Set input values for RealFlight function call.
ctipe_file_dir = 'C:/Users/rringuet/Kamodo_Data/CTIPe/Storm_201303/'
gitm_file_dir = 'D:/GITM/Storm_201303/'
waccmx_file_dir = 'D:/WACCMX/storm_201303/'
start_utcts = dt.datetime(2013, 3, 16, 0).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2013, 3, 17, 0).replace(tzinfo=dt.timezone.utc).timestamp()-1
variables = ['T_i']  # one or more variable names to retrieve
coord_sys = 'GEO'  # requested cartesian system for satellite positions

# Fly the cnofs trajectory through the simulated data chosen. See SSCWeb for the satellite names.
results_ctipe = SF.RealFlight('cnofs', start_utcts, end_utcts, 'CTIPe', ctipe_file_dir,
                              variables, coord_sys)
results_gitm = SF.RealFlight('cnofs', start_utcts, end_utcts, 'GITM', gitm_file_dir, 
                             variables, coord_sys)
results_waccmx = SF.RealFlight('cnofs', start_utcts, end_utcts, 'WACCMX', waccmx_file_dir, 
                               variables, coord_sys)

In [ ]:
# Functionalize the data.
kamodo_object = SF.O.Functionalize_TimeSeries(results_ctipe['utc_time'], 'CTIPeT_i',
                                              'K', results_ctipe['T_i'])
kamodo_object = SF.O.Functionalize_TimeSeries(results_gitm['utc_time'], 'GITMT_i',
                                              'K', results_gitm['T_i'], kamodo_object)
kamodo_object = SF.O.Functionalize_TimeSeries(results_waccmx['utc_time'], 'WACCMXT_i',
                                              'K', results_waccmx['T_i'], kamodo_object)
kamodo_object

In [ ]:
# Perform a simple ensembling analysis as an example.
kamodo_object['ModelAverage[K]'] = '(CTIPeT_i + GITMT_i + WACCMXT_i)/3.'
kamodo_object['CTIPe_diff'] = 'abs(ModelAverage - CTIPeT_i)'
kamodo_object['GITM_diff'] = 'abs(ModelAverage - GITMT_i)'
kamodo_object['WACCMX_diff'] = 'abs(ModelAverage - WACCMXT_i)'
kamodo_object

In [ ]:
# Now we can plot model and calculations on the same figure with the difference
kamodo_object.plot('CTIPeT_i', 'GITMT_i', 'WACCMXT_i', 'ModelAverage', 'CTIPe_diff', 'GITM_diff', 'WACCMX_diff')

In [ ]:
# Get CINDI data through HAPI
# HAPI sometimes complains. If that happens, just run the notebook again.
from kamodo_ccmc.readers.hapi import HAPI

server = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset = 'CNOFS_CINDI_IVM_500MS'
parameters = 'ionTemperature'
start = dt.datetime.utcfromtimestamp(start_utcts).replace(tzinfo=dt.timezone.utc).isoformat('T')
stop = dt.datetime.utcfromtimestamp(end_utcts).replace(tzinfo=dt.timezone.utc).isoformat('T')
hapiCDA = HAPI(server, dataset, parameters, start, stop)

# Add the new time series into the Kamodo object and analyze
kamodo_object = SF.O.Functionalize_TimeSeries(hapiCDA.tsarray, 'CINDIT_i', 
                                              'K', hapiCDA.ionTemperature(), 
                                              kamodo_object=kamodo_object)

kamodo_object['DataModel_DIFF[K]'] = 'abs(CINDIT_i - ModelAverage)'
kamodo_object

In [ ]:
# Now we can plot model and data on the same figure with the difference.
# Kamodo does not add data where the model data is not defined.
kamodo_object.plot('CINDIT_i', 'ModelAverage', 'DataModel_DIFF')